In [70]:
import os
from joblib import dump, load
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib as plt
from tscv import GapRollForward
from tqdm.notebook import tqdm

# --- notebook parameters: import, choose model, set hyperparameters
from sklearn.ensemble import HistGradientBoostingRegressor

DATA_PATH = '../data/sa/merged2.csv'
TRAIN_BEGIN = '2018-03-07'
TRAIN_MIN_SIZE = 24 * 365   # change for expanding window
TRAIN_MAX_SIZE = 24 * 365   # change for expanding window (np.inf)
TEST_MIN_SIZE = 24 * 7
TEST_MAX_SIZE = 24 * 7
TEST_FINAL_N = None         # set to None for rolling test window, or n to test final observations
ROLL_SIZE = 24 * 30

MODEL_SELECTION = 'hgb'
MODELS_DEFINITION = {
    'hgb': {
        'class': HistGradientBoostingRegressor,
        'kwargs': {}
    }
}
MODEL_DIR = f'../models/sa/{MODEL_SELECTION}'
# --- end notebook parameters

In [71]:

MODEL = MODELS_DEFINITION[MODEL_SELECTION]['class']
MODEL_KWARGS = MODELS_DEFINITION[MODEL_SELECTION].get('kwargs', {})
if not os.path.isdir(MODEL_DIR):
    os.makedirs(MODEL_DIR)

# extract holidays from file
holiday_df = pd.read_csv('../data/holidays2017_2024.csv', dtype='str')
holiday_df['Date'] = holiday_df['Date'].astype('datetime64[ns]').dt.date
holidays = holiday_df.loc[holiday_df['Jurisdiction'] == 'sa', ['Date', 'Holiday Name']]
holidict = {name: i+1 for i, name in enumerate(holidays['Holiday Name'].unique())}

# import and preprocess SA data
df = pd.read_csv(os.path.relpath(DATA_PATH))
df.datetime = df.datetime.astype('datetime64')
dt = df['datetime'].dt
df['year'] = dt.year
df['month'] = dt.month
df['day'] = dt.day
df['hour'] = dt.hour
df['minute'] = dt.minute
df['day_of_week'] = dt.day_of_week
df['week'] = dt.isocalendar().week
df['date'] = dt.date.astype('str')
df['holiday'] = dt.date.isin(holidays['Date']).astype('int')
X_inds = list(range(1, 8)) + list(range(11, 18)) + [19]
y_ind = 9

df_subset = df[df.datetime >= TRAIN_BEGIN]

# specify rolling training window strategy
tscv = GapRollForward(min_train_size=TRAIN_MIN_SIZE, max_train_size=TRAIN_MAX_SIZE,
                      min_test_size=TEST_MIN_SIZE, max_test_size=TEST_MAX_SIZE,
                      roll_size=ROLL_SIZE)
print(sum(1 for i in tscv.split(df_subset)), 'models to be loaded/trained')
df_subset.iloc[:, X_inds].head()

49 models to be loaded/trained


,tempc,cloud8,windk,wdir,humid,rainmm,radkjm2,year,month,day,hour,minute,day_of_week,week,holiday
14,18.0,1.0,7.0,180.0,74.0,0.0,0.0,2018,3,7,0,0,2,10,0
15,19.0,1.0,12.0,130.0,64.0,0.0,0.0,2018,3,7,1,0,2,10,0
16,19.5,0.0,13.0,120.0,55.0,0.0,0.0,2018,3,7,2,0,2,10,0
17,19.4,1.0,11.0,120.0,55.0,0.0,0.0,2018,3,7,3,0,2,10,0
18,20.0,3.0,14.0,120.0,53.0,0.0,0.0,2018,3,7,4,0,2,10,0


In [72]:
# load persisted models if they exist, otherwise train/persist new models and predict
prdfs = []
for i, (train_ind, test_ind) in tqdm(enumerate(tscv.split(df_subset))):
    if TEST_FINAL_N:
        test_ind = range(-TEST_FINAL_N, 0)

    X_train, X_test = df_subset.iloc[train_ind, X_inds], df_subset.iloc[test_ind, X_inds]
    y_train, y_test = df_subset.iloc[train_ind, y_ind], df_subset.iloc[test_ind, y_ind]

    # train or load
    begin, end = df_subset.iloc[[train_ind[0], train_ind[-1]], 0].dt.date
    model = MODEL(**MODEL_KWARGS)
    model.fit(X_train, y_train)

    # predict
    prd = model.predict(X_test)
    prdf = pd.DataFrame({'datetime': df_subset.iloc[test_ind, 0], 
                        'model': i,
                        'train_end': end,
                        'predicted': prd,
                        'net_load': y_test})
    prdfs.append(prdf)

predictions = pd.concat(prdfs)

0it [00:00, ?it/s]

In [73]:
import plotly.express as px

predictions['Residual'] = predictions['net_load'] - predictions['predicted']
predictions['Percent Error'] = predictions['Residual'] / predictions['net_load']
predictions['Absolute Percent Error'] = predictions['Percent Error'].abs() * 100
predictions['Squared Error'] = predictions['Residual'] ** 2
prediction_summary = predictions.groupby('train_end').describe()

# to display in plot title
model_type = str(MODEL).split("'")[1].split(".")[-1]
mae = predictions["Residual"].abs().mean()
mape = predictions["Absolute Percent Error"].mean()
rmse = predictions["Squared Error"].mean() ** 0.5

# massage prediction summary into long-form with desired quartiles
plot_df = prediction_summary.stack([0, 1]).loc[:, 
    ['Residual', 'Absolute Percent Error', 'Squared Error'],
    ['max', '75%', '50%', '25%', 'min']
    ].reset_index(name='value').rename(
        {'level_1': 'metric', 'level_2': 'quartile'}, axis=1)

max_mape_train_end = predictions.iloc[predictions['Absolute Percent Error'].argmax(), :]['train_end']

fig = px.line(plot_df, x="train_end", y="value", color="quartile", facet_col_wrap=1,
    facet_col="metric", #title=f'{model_type}        MAE: {mae:.2f} MAPE: {mape:.2%} RMSE {rmse:.2f}',
    template='ggplot2', width=600, height=400)
fig.update_layout(margin=dict(l=20, r=20, t=20, b=20))
fig.update_yaxes(matches=None, title=None)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.add_vline(max_mape_train_end, line_width=3, line_dash="dot", line_color="black")
fig.write_image('../plots/hgb_residuals.png')
fig.show()

In [74]:
fulldf = df[['datetime', 'radkjm2', 'pv_est']].merge(predictions, on='datetime').set_index('datetime').sort_index()

top_days = fulldf[['Squared Error']].resample('1D').sum().sort_values(
    'Squared Error', ascending=False).head(9).index
top_dates = [str(x)[:10] for x in top_days.values]
top_df = pd.concat([fulldf.loc[td] for td in top_dates]).reset_index()
top_df['date'] = top_df['datetime'].dt.date.astype('str')

line_vars = ['net_load', 'predicted', 'radkjm2', 'pv_est']
id_vars = ['datetime', 'date']
day_plot_df = top_df[line_vars + id_vars].melt(id_vars=id_vars)
day_plot_df['hour'] = day_plot_df['datetime'].dt.hour
day_plot_df['dash'] = day_plot_df['variable'].isin(['pv_est', 'radkjm2'])

fig = px.line(day_plot_df, x='hour', y='value', facet_col='date',
                 facet_col_wrap=3, color='variable', template='ggplot2',
                 width = 600, height = 400, facet_row_spacing=0.1, line_dash='dash')
bad_linenames = ['net_load, False', 'predicted, False', 'radkjm2, True', 'pv_est, True']
line_rename = {k: v for k, v in zip(bad_linenames, line_vars)}
fig.for_each_trace(lambda x: x.update(name = line_rename[x.name]))
fig.update_layout(legend_title_text='Variable', margin=dict(l=20, r=20, t=20, b=50))
fig.update_yaxes(title=None)
fig.update_xaxes(matches=None, title=None)
fig.for_each_annotation(lambda a: a.update(text=a.text.split("=")[-1]))
fig.add_annotation(showarrow=False, xanchor='center', xref='paper', 
    x=0.5, yref='paper', y=-0.13, text='Hour')
fig.write_image('../plots/hgb_residual_plots.png')
fig.show()

In [75]:
fulldf = df.merge(predictions, on='datetime').set_index('datetime').sort_index()
plotdf = fulldf[fulldf['radkjm2'] > 0].reset_index()

fig = px.scatter(plotdf, y='Residual', x='radkjm2', color_discrete_sequence=["black"],
    opacity=0.1, trendline="ols", trendline_color_override="red",
    template='ggplot2', height=400, width=600)
fig.update_layout(margin=dict(l=20, r=20, t=20, b=20))
fig.write_image('../plots/hgb_residual_pairs.png')
fig.show()